### Leer el archivo JSON usando el "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movies_casts_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("castOrder", IntegerType(), True)
])

In [0]:
movies_casts_df = spark.read.schema(movies_casts_schema).option("multiLine", True).json(
    f"{bronze_folder_path}/{v_file_date}/movie_cast.json"
)

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
movies_casts_with_columns_df = movies_casts_df.withColumnsRenamed({
    'movieId': 'movie_id',
    'personId': 'person_id',
    'characterName': 'character_name',
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'file_date': lit(v_file_date)
})

In [0]:
movies_casts_final_df = movies_casts_with_columns_df.drop('genderId', 'castOrder')

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.movies_casts")

overwrite_partition(movies_casts_final_df, "movie_silver", "movies_casts", "file_date")


In [0]:
#movies_casts_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/movies_casts")

#movies_casts_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies_casts")

movies_casts_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_casts")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/movies_casts

In [0]:
%sql
select file_date, count(1)
from movie_silver.movies_casts
group by file_date;